In [1]:
from src.gen import Gen
from src.puzzle import Nonogram
from src.state import State

# grid = [
#     [1, 1, 1, 0, 1],
#     [1, 1, 1, 0, 1],
#     [0, 0, 1, 1, 1],
#     [0, 0, 0, 1, 0],
#     [0, 0, 0, 1, 0]
# ]
# grid = [
#     [1, 1, 0, 1],
#     [0, 1, 0, 0],
#     [0, 1, 1, 0],
#     [1, 0, 0, 1]
# ]
# grid = [
#     [1, 0, 1],
#     [0, 0, 1],
#     [0, 1, 1]
# ]
# grid = [
#     [1, 0],
#     [0, 1]
# ]
# init = State(
#     len(grid), 
#     num=Gen.gen_grid_num(grid)
# )
init = State(size=7)

from src.search import DFS

puzzle = Nonogram(init)
print(DFS(puzzle).path())


Input:
#  .  #  #  .  #  #  #    1 2 3 
.  .  #  .  .  .  #  .    1 1 
#  #  .  #  #  .  #  #    2 2 2 
.  .  #  .  .  .  #  #    1 2 
#  #  #  #  #  .  #  #    5 2 
#  .  .  .  #  .  .  #    1 1 1 
.  .  .  #  #  .  .  .    2 
#  .  .  .  #  .  .  #    1 1 1 

1  1  2  1  1  1  5  1  
1  1  2  1  4        4  
2        1           1  
1        1              



KeyboardInterrupt: 